In [49]:
import anthropic
import json
import re
import os
from rich.console import Console
from dotenv import load_dotenv
from jinja2 import Environment, FileSystemLoader

## Setup

In [32]:
load_dotenv("../.env")

console = Console(style="bold green")

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=ANTHROPIC_API_KEY
)

## Test Tool Avaialability

In [34]:
response = client.beta.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1000,
    messages=[{
        "role": "user",
        "content": "What tools do you have available?"
    }],
    mcp_servers=[
        {
            "type": "url",
            "url": "https://mcp.deepsense.ai/icd10_codes/mcp",
            "name": "icd10-codes"
        }
    ],
    tools=[
        {
            "type": "mcp_toolset",
            "mcp_server_name": "icd10-codes"
        }
    ],
    betas=["mcp-client-2025-11-20"]
)

In [35]:
console.print(response.content[0].text)

I have access to specialized tools for working with ICD-10 medical coding systems. Here's what I can do:

## ICD-10 Code Tools

1. **Search Codes** (`icd10-codes_search_codes`)
   - Search for diagnosis (ICD-10-CM) or procedure (ICD-10-PCS) codes
   - Search by code prefix or by description text
   - Filter for HIPAA-valid codes only
   - Limit results as needed

2. **Lookup Specific Code** (`icd10-codes_lookup_code`)
   - Get full details for an exact ICD-10 code
   - Works for both diagnosis and procedure codes
   - Returns descriptions and HIPAA transaction validity status

3. **Validate Code** (`icd10-codes_validate_code`)
   - Check if a code exists and is valid
   - Verify HIPAA transaction eligibility
   - Identifies category headers vs. billable codes

4. **Get Code Hierarchy** (`icd10-codes_get_hierarchy`)
   - Explore all codes under a category
   - See parent-child relationships
   - Useful for finding related codes in a disease family

5. **Get Codes by Category** (`icd10-codes_get_by_category`)
   - Browse diagnosis codes by ICD-10-CM chapter (e.g., 'E' for Endocrine)
   - Or by specific category (e.g., 'E11' for Type 2 Diabetes)
   - Filter for HIPAA-valid codes

6. **Get Codes by Body System** (`icd10-codes_get_by_body_system`)
   - Find procedure codes for specific anatomical systems
   - Filter by section (Medical/Surgical, Rehabilitation, etc.)
   - Useful for procedure-related queries

These tools help with medical coding research, validation, and exploration of the ICD-10 classification system. How
can I help you with ICD-10 codes?

## Example Usage

In [64]:
# Load the Jinja2 template
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('prompt_template.md.jinja2')

# Render the template with actual values
clinical_note = "Patient is a 65-year-old male with history of hypertension and diabetes mellitus type 2. Presents to the ED with chest pain radiating to left arm, associated with diaphoresis. ECG shows ST elevation in leads II, III, and aVF. Troponin elevated at 2.5 ng/mL."
clinical_phrase = "acute inferior myocardial infarction"

prompt_template = template.render(
    clinical_note=clinical_note,
    clinical_phrase=clinical_phrase
)

console.print(prompt_template, style="black on white")

You will be acting as a medical coding specialist. Your task is to identify the most specific billable ICD-10 code 
for a clinical phrase based on the context provided in a clinical note.

Here is the full clinical note for context:

<clinical_note>
Patient is a 65-year-old male with history of hypertension and diabetes mellitus type 2. Presents to the ED with 
chest pain radiating to left arm, associated with diaphoresis. ECG shows ST elevation in leads II, III, and aVF. 
Troponin elevated at 2.5 ng/mL.
</clinical_note>

Here is the specific phrase that needs to be coded:

<clinical_phrase>
acute inferior myocardial infarction
</clinical_phrase>

Important guidelines for ICD-10 coding:
- ICD-10 codes must be as specific as possible based on the available clinical information
- Only billable codes should be provided (typically these are codes at the highest level of specificity, not 
category codes)
- The code must be consistent with all information in the clinical note
- Use the context from the full clinical note to determine laterality, severity, episode of care, and other 
specificity requirements
- If the phrase is ambiguous, use the clinical note context to clarify the most appropriate code
- Do not code suspected, possible, or rule-out conditions unless explicitly documented as diagnoses

Before providing your final answer, use the <scratchpad> section to:
1. Identify the key clinical concepts in the phrase
2. Consider relevant context from the clinical note (laterality, chronicity, severity, etc.)
3. Determine the most specific ICD-10 code that captures the condition
4. Verify that the code is billable (not a category or subcategory code)
5. Confirm the code is consistent with all information in the clinical note

After your scratchpad analysis, provide your final answer in the following format:

<answer>
<icd10_code>[The specific ICD-10 code]</icd10_code>
<description>[Brief description of what the code represents]</description>
<justification>[Explanation of why this is the most appropriate and specific billable code based on the clinical 
phrase and note context]</justification>
</answer>

Your final answer should contain only the ICD-10 code, its description, and the justification. Do not include the 
scratchpad content in your final answer.

In [39]:
no_tool_response = client.beta.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=1024,
    betas=["structured-outputs-2025-11-13"],
    messages=[
        {
            "role": "user",
            "content": prompt_template}
    ],
    output_format={
        "type": "json_schema",
        "schema": {
            "type": "object",
            "properties": {
                "ICD10": {"type": "string"},
                "description": {"type": "string"},
                "justification": {"type": "string"}
            },
            "required": ["ICD10", "description", "justification"],
            "additionalProperties": False
        }
    },
)

In [40]:
res = json.loads(no_tool_response.content[0].text)
console.print("\n[bold green]Structured Output:[/bold green]")
console.print(res)

Structured Output:

{
    'ICD10': 'I21.19',
    'description': 'ST elevation (STEMI) myocardial infarction involving other coronary artery of inferior wall',
    'justification': "The clinical phrase 'acute inferior myocardial infarction' combined with the clinical context
showing ST elevation in leads II, III, and aVF (indicating inferior wall involvement) and elevated troponin 
confirms an acute ST elevation myocardial infarction (STEMI) of the inferior wall. I21.19 is the most specific 
billable code for STEMI of the inferior wall when the specific coronary artery is not documented or when it 
involves other coronary arteries not specifically listed in other I21.1x codes. This represents the initial episode
of care as documented in the ED presentation."
}

## Exmample Usage with Tools

In [68]:
# Load the Jinja2 template
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('prompt_template_tools.md.jinja2')

# Render the template with actual values
clinical_note = "Patient is a 65-year-old male with history of hypertension and diabetes mellitus type 2. Presents to the ED with chest pain radiating to left arm, associated with diaphoresis. ECG shows ST elevation in leads II, III, and aVF. Troponin elevated at 2.5 ng/mL."
clinical_phrase = "acute inferior myocardial infarction"

prompt_template = template.render(
    clinical_note=clinical_note,
    clinical_phrase=clinical_phrase
)

In [69]:
# Example: Extract ICD10 code with tool usage enabled
# Claude can use the ICD10 tools to validate and lookup codes

response_with_tools = client.beta.messages.create(
    model="claude-sonnet-4-5",
    max_tokens=2048,
    messages=[{
        "role": "user",
        "content": prompt_template
    }],
    mcp_servers=[
        {
            "type": "url",
            "url": "https://mcp.deepsense.ai/icd10_codes/mcp",
            "name": "icd10-codes"
        }
    ],
    tools=[
        {
            "type": "mcp_toolset",
            "mcp_server_name": "icd10-codes"
        }
    ],
    betas=["mcp-client-2025-11-20"]
)

print(response_with_tools.content[-1].text)

<answer>
<icd10_code>I21.19</icd10_code>
<description>ST elevation (STEMI) myocardial infarction involving other coronary artery of inferior wall</description>
<justification>This is the most appropriate and specific billable code for acute inferior myocardial infarction based on the clinical presentation. The clinical note confirms this is an acute STEMI with ST elevation in leads II, III, and aVF (which are the classic ECG leads showing inferior wall MI), along with elevated troponin confirming acute myocardial infarction. Code I21.19 is used because while the clinical documentation specifies the location (inferior wall) and confirms ST elevation, it does not specifically identify which coronary artery is involved. The parent category code I21.1 is not billable; therefore, I21.19 (involving other coronary artery of inferior wall) is the most specific billable code available for this clinical scenario.</justification>
</answer>


In [75]:
console.print(response_with_tools.content, style="white on black")

[
    BetaTextBlock(
        citations=None,
        text='<scratchpad>\n1. Key clinical concepts in the phrase:\n   - Acute myocardial infarction\n   - 
Inferior location (correlates with ST elevation in leads II, III, and aVF per the clinical note)\n\n2. Relevant 
context from clinical note:\n   - Patient presents to ED (initial encounter)\n   - ECG shows ST elevation in leads 
II, III, and aVF (indicates inferior wall MI)\n   - This is an STEMI (ST-elevation myocardial infarction)\n   - 
Elevated troponin confirms acute MI\n   - This appears to be the initial episode\n\n3. Need to search for specific 
codes for acute inferior myocardial infarction\n\n4. Must verify the code is billable and at the highest level of 
specificity\n\n5. Need to ensure the code matches the clinical presentation\n</scratchpad>\n\nLet me search for the
appropriate ICD-10 codes for acute inferior myocardial infarction.',
        type='text'
    ),
    BetaMCPToolUseBlock(
        id='mcptoolu_01VLGnncH2FxZXASQBLM9VmB',
        input={
            'query': 'acute inferior myocardial infarction',
            'search_by': 'description',
            'code_type': 'diagnosis',
            'valid_for_hipaa_only': True
        },
        name='search_codes',
        server_name='icd10-codes',
        type='mcp_tool_use'
    ),
    BetaMCPToolResultBlock(
        content=[
            BetaTextBlock(
                citations=None,
                text='{"count":20,"results":[{"code":"I21.9","short_description":"Acute myocardial infarction, 
unspecified","long_description":"Acute myocardial infarction, 
unspecified","valid_for_hipaa_transactions":true,"chapter":"I","category":"I21"},{"code":"I22.1","short_description
":"Subsequent STEMI of inferior wall","long_description":"Subsequent ST elevation (STEMI) myocardial infarction of 
inferior 
wall","valid_for_hipaa_transactions":true,"chapter":"I","category":"I22"},{"code":"I24.0","short_description":"Acut
e coronary thrombosis not resulting in myocardial infrc","long_description":"Acute coronary thrombosis not 
resulting in myocardial 
infarction","valid_for_hipaa_transactions":true,"chapter":"I","category":"I24"},{"code":"I25.2","short_description"
:"Old myocardial infarction","long_description":"Old myocardial 
infarction","valid_for_hipaa_transactions":true,"chapter":"I","category":"I25"},{"code":"I21.A1","short_description
":"Myocardial infarction type 2","long_description":"Myocardial infarction type 
2","valid_for_hipaa_transactions":true,"chapter":"I","category":"I21"},{"code":"I21.A9","short_description":"Other 
myocardial infarction type","long_description":"Other myocardial infarction 
type","valid_for_hipaa_transactions":true,"chapter":"I","category":"I21"},{"code":"I23.8","short_description":"Oth 
current complications following AMI","long_description":"Other current complications following acute myocardial 
infarction","valid_for_hipaa_transactions":true,"chapter":"I","category":"I23"},{"code":"I21.19","short_description
":"STEMI involving oth coronary artery of inferior wall","long_description":"ST elevation (STEMI) myocardial 
infarction involving other coronary artery of inferior 
wall","valid_for_hipaa_transactions":true,"chapter":"I","category":"I21"},{"code":"I23.0","short_description":"Hemo
pericardium as current complication following AMI","long_description":"Hemopericardium as current complication 
following acute myocardial 
infarction","valid_for_hipaa_transactions":true,"chapter":"I","category":"I23"},{"code":"I21.4","short_description"
:"Non-ST elevation (NSTEMI) myocardial infarction","long_description":"Non-ST elevation (NSTEMI) myocardial 
infarction","valid_for_hipaa_transactions":true,"chapter":"I","category":"I21"},{"code":"I21.B","short_description"
:"Myocardial infarction with coronary microvascular dysfnct","long_description":"Myocardial infarction with 
coronary microvascular 
dysfunction","valid_for_hipaa_transactions":true,"chapter":"I","category":"I21"},{"code":"I22.2","s